# NLP CA4 - Q1
### Ali Khoramfar - 810102129

## Install Packages

In [3]:
!pip install evaluate
!pip install --upgrade accelerate
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.1 MB/s eta 0:00:00


## Import Packages

In [4]:
from datasets import load_dataset
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from accelerate import Accelerator

from datasets import load_metric
import time
import logging
import evaluate
import torch
from torchinfo import summary

from peft import get_peft_model, LoraConfig,PromptTuningConfig, TaskType


2024-05-27 14:56:00.477843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 14:56:00.477939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 14:56:00.598295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Multi NLI Dataset

In [5]:
dataset = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

### Dataset Structure

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})


In [7]:
# 10% of dataset
train_subset = dataset['train'].shuffle(seed=42).select(range(int(0.1 * len(dataset['train']))))

### Tokenization

In [8]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

def preprocess_function(examples):
    return tokenizer(
        examples['premise'], 
        examples['hypothesis'], 
        truncation=True, 
        padding='max_length', 
        max_length=128
    )

encoded_train = train_subset.map(preprocess_function, batched=True)
encoded_valid_matched = dataset['validation_matched'].map(preprocess_function, batched=True)
encoded_valid_mismatched = dataset['validation_mismatched'].map(preprocess_function, batched=True)

encoded_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
encoded_valid_matched.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
encoded_valid_mismatched.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Map:   0%|          | 0/39270 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

### Load Roberta-Large Model

In [9]:
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training Arguments

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=100000,
    report_to=[],
)

In [11]:
metric = evaluate.load("accuracy", trust_remote_code=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

accelerator = Accelerator()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid_matched,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train the Model  - roberta-large

In [18]:
start_time = time.time()
trainer.train()
end_time = time.time()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.510700,0.473953,0.840652
2,0.310800,0.449588,0.878859
3,0.218100,0.618880,0.877534


## Report

In [35]:
summary(model, input_size=(1, 128), dtypes=[torch.long])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 3]                    --
├─RobertaModel: 1-1                                          [1, 128, 1024]            --
│    └─RobertaEmbeddings: 2-1                                [1, 128, 1024]            --
│    │    └─Embedding: 3-1                                   [1, 128, 1024]            51,471,360
│    │    └─Embedding: 3-2                                   [1, 128, 1024]            1,024
│    │    └─Embedding: 3-3                                   [1, 128, 1024]            526,336
│    │    └─LayerNorm: 3-4                                   [1, 128, 1024]            2,048
│    │    └─Dropout: 3-5                                     [1, 128, 1024]            --
│    └─RobertaEncoder: 2-2                                   [1, 128, 1024]            --
│    │    └─ModuleList: 3-6                                  --             

In [22]:
num_parameters = model.num_parameters()
print(f"Number of Parameters: {num_parameters}")

Number of Parameters: 355362819


In [23]:
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 5570.118634223938 seconds


In [24]:
eval_results_matched = trainer.evaluate(encoded_valid_matched)
eval_results_mismatched = trainer.evaluate(encoded_valid_mismatched)

print(f"Validation matched accuracy: {eval_results_matched['eval_accuracy']}")
print(f"Validation mismatched accuracy: {eval_results_mismatched['eval_accuracy']}")

Validation matched accuracy: 0.8775343861436576
Validation mismatched accuracy: 0.8746948738812043


In [25]:
model.save_pretrained("/kaggle/working/roberta-large-nli")
tokenizer.save_pretrained("/kaggle/working/roberta-large-nli")

('/kaggle/working/roberta-large-nli/tokenizer_config.json',
 '/kaggle/working/roberta-large-nli/special_tokens_map.json',
 '/kaggle/working/roberta-large-nli/vocab.json',
 '/kaggle/working/roberta-large-nli/merges.txt',
 '/kaggle/working/roberta-large-nli/added_tokens.json')

In [21]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [28]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

## Train the Model  - LoRa

In [40]:
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,  
    lora_alpha=32,  
    lora_dropout=0.1  
)

model = get_peft_model(model, lora_config)

training_args_lora = TrainingArguments(
    output_dir="./results-lora",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs-lora',
    logging_steps=10,
    save_steps=100000,
    report_to=[],  
)

trainer_lora = Trainer(
    model=model,
    args=training_args_lora,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid_matched,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [41]:
start_time = time.time()
trainer_lora.train()
end_time = time.time()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.511900,0.418702,0.846663
2,0.438200,0.384544,0.862150
3,0.471000,0.366670,0.867652


## Report

In [42]:
summary(model, input_size=(1, 128), dtypes=[torch.long])

Layer (type:depth-idx)                                                      Output Shape              Param #
PeftModelForSequenceClassification                                          [1, 3]                    --
├─LoraModel: 1-1                                                            [1, 3]                    --
│    └─RobertaForSequenceClassification: 2-1                                --                        --
│    │    └─RobertaModel: 3-1                                               [1, 128, 1024]            355,096,576
│    │    └─ModulesToSaveWrapper: 3-2                                       [1, 3]                    2,105,350
Total params: 357,201,926
Trainable params: 1,839,107
Non-trainable params: 355,362,819
Total mult-adds (M): 356.15
Input size (MB): 0.00
Forward/backward pass size (MB): 331.75
Params size (MB): 1424.60
Estimated Total Size (MB): 1756.35

In [43]:
num_parameters = model.num_parameters()
print(f"Number of Parameters (LoRA): {num_parameters}")

Number of Parameters (LoRA): 357201926


In [44]:
training_time = end_time - start_time
print(f"Training time (LoRA): {training_time} seconds")

Training time (LoRA): 4064.483451604843 seconds


In [45]:
eval_results_matched = trainer_lora.evaluate(encoded_valid_matched)
eval_results_mismatched = trainer_lora.evaluate(encoded_valid_mismatched)

print(f"Validation matched accuracy (LoRA): {eval_results_matched['eval_accuracy']}")
print(f"Validation mismatched accuracy (LoRA): {eval_results_mismatched['eval_accuracy']}")

Validation matched accuracy (LoRA): 0.8676515537442689
Validation mismatched accuracy (LoRA): 0.8668633034987795


In [46]:
# Save the trained model
model.save_pretrained("/kaggle/working/roberta-large-nli-lora")
tokenizer.save_pretrained("/kaggle/working/roberta-large-nli-lora")

('/kaggle/working/roberta-large-nli-lora/tokenizer_config.json',
 '/kaggle/working/roberta-large-nli-lora/special_tokens_map.json',
 '/kaggle/working/roberta-large-nli-lora/vocab.json',
 '/kaggle/working/roberta-large-nli-lora/merges.txt',
 '/kaggle/working/roberta-large-nli-lora/added_tokens.json')

In [47]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [49]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): M

## Train the Model  - Prompt Tuning

In [12]:

model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=30,  
)

model = get_peft_model(model, prompt_tuning_config)

for param in model.parameters():
    param.requires_grad = False
for param in model.get_input_embeddings().parameters():
    param.requires_grad = True
    
    
training_args_prompt = TrainingArguments(
    output_dir="./results-prompt-tuning",
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs-prompt-tuning',
    logging_steps=10,
    save_steps=100000,
    report_to=[],  
)

trainer_prompt = Trainer(
    model=model,
    args=training_args_prompt,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid_matched,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
start_time = time.time()
trainer_prompt.train()
end_time = time.time()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.193300,1.126348,0.354457
2,1.173000,1.124661,0.354457
3,1.134500,1.124166,0.354457


## Report

In [14]:
summary(model, input_size=(1, 128), dtypes=[torch.long])

Layer (type:depth-idx)                                            Output Shape              Param #
PeftModelForSequenceClassification                                [1, 3]                    --
├─Embedding: 1-1                                                  [1, 128, 1024]            51,471,360
├─ModuleDict: 1-2                                                 --                        --
│    └─PromptEmbedding: 2-1                                       [1, 30, 1024]             --
│    │    └─Embedding: 3-1                                        [1, 30, 1024]             (30,720)
├─RobertaForSequenceClassification: 1-3                           [1, 3]                    --
│    └─RobertaModel: 2-2                                          [1, 158, 1024]            --
│    │    └─RobertaEmbeddings: 3-2                                [1, 158, 1024]            52,000,768
│    │    └─RobertaEncoder: 3-3                                   [1, 158, 1024]            (302,309,376)
│    └─Modul

In [15]:
num_parameters = model.num_parameters()
print(f"Number of Parameters (Prompt Tuning): {num_parameters}")

Number of Parameters (Prompt Tuning): 356415494


In [16]:
training_time = end_time - start_time
print(f"Training time (Prompt Tuning): {training_time} seconds")

Training time (Prompt Tuning): 4466.86380147934 seconds


In [17]:
eval_results_matched = trainer_prompt.evaluate(encoded_valid_matched)
eval_results_mismatched = trainer_prompt.evaluate(encoded_valid_mismatched)

print(f"Validation matched accuracy (Prompt Tuning): {eval_results_matched['eval_accuracy']}")
print(f"Validation mismatched accuracy (Prompt Tuning): {eval_results_mismatched['eval_accuracy']}")

Validation matched accuracy (Prompt Tuning): 0.3544574630667346
Validation mismatched accuracy (Prompt Tuning): 0.3522172497965826


In [18]:
model.save_pretrained("/kaggle/working/roberta-large-nli-prompt-tuning")
tokenizer.save_pretrained("/kaggle/working/roberta-large-nli-prompt-tuning")

('/kaggle/working/roberta-large-nli-prompt-tuning/tokenizer_config.json',
 '/kaggle/working/roberta-large-nli-prompt-tuning/special_tokens_map.json',
 '/kaggle/working/roberta-large-nli-prompt-tuning/vocab.json',
 '/kaggle/working/roberta-large-nli-prompt-tuning/merges.txt',
 '/kaggle/working/roberta-large-nli-prompt-tuning/added_tokens.json')

In [21]:
model.config

RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [22]:
print(model)

PeftModelForSequenceClassification(
  (base_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaS